In [23]:
import pandas
import numpy
import scipy
import string
import numpy as np
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
file = open("./marktwain/treated/combined.txt", "r")
lines = file.readlines()
filestring = " ".join(lines)

In [24]:
stringArray = filestring.split(" ")
chunksArr = np.array_split(stringArray, 10000)

In [25]:
from gensim.models import doc2vec
mtArray = []
for item in chunksArr:
    global mtArray
    item = [item.tolist(),['marktwain']]
    mtArray.append(item)
sentenceArray = []
for item in mtArray:
    sentence = doc2vec.LabeledSentence(
    words=item[0], tags=item[1])
    sentenceArray.append(sentence)

In [26]:
len(sentenceArray)

10000

In [27]:
file = open("./janeausten/ja_COMBINED_treated.txt", "r")
lines2 = file.readlines()
filestring2 = " ".join(lines2)

In [28]:
stringArray2 = filestring2.split(" ")
chunksArr2 = np.array_split(stringArray2, 10000)

In [29]:
jaArray = []
for item2 in chunksArr2:
    global jaArray
    item2 = [item2.tolist(),['janeausten']]
    jaArray.append(item2)
%store jaArray

sentenceArray2 = []
for item in jaArray:
    sentence = doc2vec.LabeledSentence(
    words=item[0], tags=item[1])
    sentenceArray.append(sentence)
len(sentenceArray)

Stored 'jaArray' (list)


20000

In [30]:
#building vocabulary with combined marktwain and jane austen labeled sentences
combinedmodel = doc2vec.Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)
combinedmodel.build_vocab(sentenceArray)

In [ ]:
from random import shuffle
for epoch in range(10):
    combinedmodel.train(sentenceArray, total_examples=combinedmodel.corpus_count, epochs=combinedmodel.iter)
    shuffle(sentenceArray)
    combinedmodel.alpha -= 0.002  # decrease the learning rate
    combinedmodel.min_alpha = combinedmodel.alpha  # fix the learning rate, no decay

In [ ]:
combinedmodel.docvecs[1]

In [36]:
inputwords = []
for labeledsentence in sentenceArray:
    inputwords.append(labeledsentence.words)
len(inputwords)

20000

In [40]:
train = []
for sentence in inputwords:
    iv = combinedmodel.infer_vector(sentence)
    train.append(iv)
len(train)

20000

In [41]:
import pandas as pd
X = []
yone = np.ones(10000)
yzero = np.zeros(10000)
yzero = pd.DataFrame(yzero)
yone = pd.DataFrame(yone)
Y = pd.concat([yone,yzero])
Y.head()

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [42]:
import pandas as pd
train = pd.DataFrame(train)
#logistic regression
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression()
logreg.fit(train, Y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
%store logreg


Stored 'logreg' (LogisticRegression)


In [ ]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences